In [30]:
import pandas as pd
import numpy as np

In [31]:
FILE_PATH = 'day_2_data.txt'

In [39]:
keymap_dict = {
    'red': 12,
    'green': 13,
    'blue': 14,
}

answer = 0
with open(FILE_PATH, mode='r') as file:
    for line in file:
        possible = True
        line = line.strip()
        id_, line = line.split(':')
        for balls in line.split(';'):
            for ball in balls.split(','):
                n, color = ball.split()
                if int(n) > keymap_dict.get(color, 0):
                    possible = False
                    print(n, color)
        if possible:
            print(balls)
            print(color)
            print(n)
            answer += int(id_.split()[-1])
            print(int(id_.split()[-1]))
print(answer)

 2 blue, 8 green
green
8
2
 8 blue, 2 green, 8 red
red
8
3
 7 blue, 1 red, 3 green
green
3
8
 10 blue, 3 green, 6 red
red
6
11
 3 red, 2 green
green
2
12
 4 green, 3 red, 2 blue
blue
2
15
 2 green, 1 blue, 1 red
red
1
16
 8 red, 14 blue, 2 green
green
2
19
 11 blue, 1 red, 3 green
green
3
21
 5 red, 3 blue, 1 green
green
1
22
 11 blue, 9 green, 7 red
red
7
26
 2 green, 5 blue, 2 red
red
2
28
 7 green, 14 blue, 3 red
red
3
29
 3 blue, 7 red
red
7
31
 2 red, 10 blue, 3 green
green
3
35
 7 blue, 2 green, 5 red
red
5
36
 6 blue, 1 red, 2 green
green
2
38
 2 green
green
2
40
 2 red, 2 green, 1 blue
blue
1
42
 3 blue, 1 green
green
1
43
 9 red, 1 green
green
1
46
 7 blue, 6 red
red
6
50
 2 red, 12 blue, 8 green
green
8
51
 9 red, 5 green, 8 blue
blue
8
52
 5 blue, 5 green
green
5
54
 3 blue, 2 green, 7 red
red
7
56
 3 blue, 8 red, 2 green
green
2
57
 3 blue, 1 green, 1 red
red
1
59
 10 green, 1 red, 4 blue
blue
4
60
 2 red, 4 green
green
4
61
 13 green, 9 blue
blue
9
62
 1 blue, 8 green, 3 r

In [ ]:
import sys